# Practice using an LLM for Data Extraction

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from datasets import load_dataset

from dotenv import load_dotenv

import json
from time import sleep
import pandas as pd
import os

from azure.identity import (
    DefaultAzureCredential,
)

from azure.identity import AzureAuthorityHosts
from azure.keyvault.secrets import SecretClient

### Create your Azure OpenAI Resource and Key Vault

_If you have your resource from the last exercise, you don't need to complete the following steps._

Navigate to the [Azure Portal](https://portal.azure.com/#home) or [US Gov Azure Portal](https://portal.azure.us/#home) and login using your account. Next you're going to create an Azure OpenAI resource, create a new resource group and use any unique name for the resource's name.  
Once the resource is created, you need to open [Azure AI Foundry](https://ai.azure.com/) or [Azure OpenAI Studio](https://ai.azure.us/) to deploy the model. Navigate to deployments, press deploy model and select gpt-4o-mini.  Make sure to increase your rate limit, or tokens per minute (around 700k should be sufficient)

Once that is created, copy the key (Found under Resource Management > Keys and Endpoints) and create a new key vault. Assign the same resource group as your Azure OpenAI resource and again pick a unique name for the key vault name.  

Once the key vault is created, make a new secret with the API key.  

### Connect to Azure OpenAI
First, we need to login to our Azure account.  
<br>
If you are using a Gov account:<br>
```
az cloud set --name AzureUSGovernment  
az login  
az account set --subscription="your subscription"  
```
If you are using a commercial account:<br>
```
az login  
az account set --subscription="your subscription" 
```
Two things are needed to connect to your Azure OpenAI resource
- Your API key
- Your Endpoint  
  
For the API Key, we are going to connect to the key vault we just made to insert the key. For this, you'll need to change the URL below to match your key vault's URL.  
Next, we will insert the endpoint URL from our Azure OpenAI resource.

Our `azure_client` is where we are calling the LLM and connecting to the model we deployed. Other parameters can be passed in, like timeout or max_retires.

Note: In the block below we inserted `credential = DefaultAzureCredential(authority=AzureAuthorityHosts.AZURE_GOVERNMENT)`. This is because when we use our FedAIRS account we are connected to the US Government cloud, rather than the regular commercial cloud.

In [ ]:
load_dotenv()

credential = DefaultAzureCredential(authority=AzureAuthorityHosts.AZURE_GOVERNMENT)

secret_client = SecretClient(vault_url=os.getenv('KEY_VAULT_URL'), credential=credential)
deployment = os.getenv('DEPLOYMENT')
endpoint_url = os.getenv('AZURE_OPENAI_ENDPOINT')
api_version = os.getenv('API_VERSION')
api_key = secret_client.get_secret(os.getenv('SECRET_NAME')).value


azure_client = AzureChatOpenAI(
                api_key=api_key
                ,api_version=api_version
                ,azure_endpoint=endpoint_url
                ,deployment_name=deployment
                ,temperature=0
                ,max_tokens=4000
                ,model_kwargs={"response_format": {"type": "json_object"}}
)

### Named Entity Recognition (NER)

Named Entity Recognition, or NER, is a popular technique in natural language processing (NLP) that aims to identify and classify named entities in text into predefined categories. The main goals of NER are:

* To automatically extract structured information from unstructured text
* To categorize entities into predefined types (e.g. person names, organizations, locations)
* To enable machines to understand and process key information in text  

In this exercise we are going to attempt a simpler form of NER with a small sample of data.

### The Data

Four our data, we will be using stories created by an LLM (code for this in`create_extraction_dataset.ipynb`).  

The goal is to extract the fields we prompted the LLM to use in the story:
* FirstName
* LastName
* Title
* BirthDate
* EmailAddress
* DepartmentName  

Since we used a random sample of 100 in the last exercise, we will use the full dataset we created. We will also take out any of the unused columns for simplicity.


In [3]:
stories = pd.read_csv('./data/DimEmployeeStories.csv')
stories = stories[["FirstName", "LastName", "Title", "BirthDate", "EmailAddress", "DepartmentName", "ShakespeareStory", "MarvelStory", "FairyTaleStory"]]
stories_array = stories.to_numpy()
stories.head()

,FirstName,LastName,Title,BirthDate,EmailAddress,DepartmentName,ShakespeareStory,MarvelStory,FairyTaleStory
0,Michael,DeVoe,Sales Store Manager,1966-01-21,stefen0@contoso.com,Production,"In fair Verona, where commerce doth thrive, th...","In the bustling metropolis of Production City,...",In a kingdom where the finest wares were craft...
1,Jerry,Orman,Sales Store Manager,1971-01-26,laura0@contoso.com,Production,"In fair Verona, where commerce doth thrive, th...","In the bustling metropolis of Contoso City, Je...","In a bustling kingdom of commerce, there lived..."
2,Janet,Galore,Sales Store Manager,1965-04-17,wanida0@contoso.com,Marketing,"In fair Verona, where commerce doth thrive, th...","In the bustling metropolis of Marketing City, ...",In a kingdom where the art of persuasion reign...
3,Christen,Anderson,Sales State Manager,1970-12-15,paul0@contoso.com,Production,"In fair production's realm, where dreams take ...","In the bustling metropolis of Production City,...",In a kingdom where the art of production thriv...
4,Humberto,Acevedo,Sales Region Manager,1965-01-23,Humberto@contoso.com,Tool Design,"In fair Verona, where we lay our scene, there ...","In the bustling metropolis of Contoso, Humbert...",In a kingdom where tools of great design were ...


We are going to create three prompts to extract the data from the stories and add them to our dataframe.


In [4]:
shakespeare_extraction_prompt = PromptTemplate.from_template(

template = '''
### Instructions:
Your task is to extract the Employee Details from the fake and not real Shakesperean style story provided. Do not extract any other information from the story, nor falsify data if it is not in the story. All of the BirthDates must be recorded as YYYY-MM-DD not in any other manner.
If the information is not present in the story, please return it as "Not Found".
Return your results in JSON format.

### Story:
{story}


### Return JSON:
{{
    "ExtractedFirstName": "<First Name>",
    "ExtractedLastName": "<Last Name>",
    "ExtractedTitle": "<Employee Job Title>",
    "ExtractedBirthDate": "<YYYY-MM-DD>",
    "ExtractedEmailAddress": "<Employee Email Address>",
    "ExtractedDepartmentName": "<Employee Department Name>",
}}
'''
)

In [5]:
shakespeare_stories_chain = (
    shakespeare_extraction_prompt
    | azure_client
    | StrOutputParser()
)

In [6]:
marvel_extraction_prompt = PromptTemplate.from_template(

template = '''
### Instructions:
Your task is to extract the Employee Details from the Marvel story provided. Do not extract any other information from the story, nor falsify data if it is not in the story. All of the BirthDates must be recorded as YYYY-MM-DD not in any other manner.
If the information is not present in the story, please return it as "Not Found".
Return your results in JSON format.

### Story:
{story}


### Return JSON:
{{
    "ExtractedFirstName": "<First Name>",
    "ExtractedLastName": "<Last Name>",
    "ExtractedTitle": "<Employee Job Title>",
    "ExtractedBirthDate": "<YYYY-MM-DD>",
    "ExtractedEmailAddress": "<Employee Email Address>",
    "ExtractedDepartmentName": "<Employee Department Name>",
}}
'''
)

In [7]:
marvel_stories_chain = (
    marvel_extraction_prompt
    | azure_client
    | StrOutputParser()
)

In [8]:
fairytale_extraction_prompt = PromptTemplate.from_template(

template = '''
### Instructions:
Your task is to extract the Employee Details from the Fairytale story provided. Do not extract any other information from the story, nor falsify data if it is not in the story. All of the BirthDates must be recorded as YYYY-MM-DD not in any other manner.
If the information is not present in the story, please return it as "Not Found".
Return your results in JSON format.

### Story:
{story}


### Return JSON:
{{
    "ExtractedFirstName": "<First Name>",
    "ExtractedLastName": "<Last Name>",
    "ExtractedTitle": "<Employee Job Title>",
    "ExtractedBirthDate": "<YYYY-MM-DD>",
    "ExtractedEmailAddress": "<Employee Email Address>",
    "ExtractedDepartmentName": "<Employee Department Name>",
}}
'''
)

In [9]:
fairytale_stories_chain = (
    fairytale_extraction_prompt
    | azure_client
    | StrOutputParser()
)

### Generating results

To evaluate each story type independently, we will make three separate dataframes: shakespeare, marvel, fairytale. From there, we will use the LLM to extract the data from the story.

In [10]:
shakespeare = stories[["FirstName", "LastName", "Title", "BirthDate", "EmailAddress", "DepartmentName", "ShakespeareStory"]]
marvel = stories[["FirstName", "LastName", "Title", "BirthDate", "EmailAddress", "DepartmentName", "MarvelStory"]]
fairytale = stories[["FirstName", "LastName", "Title", "BirthDate", "EmailAddress", "DepartmentName", "FairyTaleStory"]]

In [11]:
async def runChain(text_list, chain):
    texts = [{"story": text} for text in text_list]
    return await chain.abatch(texts)


In [12]:
shakespeare_stories = [text for text in shakespeare.ShakespeareStory]
marvel_stories = [text for text in marvel.MarvelStory]
fairytale_stories = [text for text in fairytale.FairyTaleStory]

### OpenAI Content Managment Policy

Because we asked the LLM to create a story in the Shakespearean style, OpenAI believed that this is Protected Material for Text. From the [website](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/content-filter?tabs=warning%2Cuser-prompt%2Cpython-new) "Protected material text describes known text content (for example, song lyrics, articles, recipes, and selected web content) that can be outputted by large language models." We will skip over the Shakespearean stories and focus on the Marvel and Fairytale ones.

In [13]:
# shakespeare_results = await runChain(shakespeare_stories, shakespeare_stories_chain)
# shakespeare['results'] = shakespeare_results
# shakespeare_results_df = pd.json_normalize(shakespeare.results.apply(json.loads))
# shakespeare_final = pd.concat([shakespeare, shakespeare_results_df], axis=1)
# shakespeare_final.drop(columns=['results'], inplace=True)
# shakespeare_final.head()

In [14]:
marvel_results = await runChain(marvel_stories, marvel_stories_chain)
marvel['results'] = marvel_results
marvel_results_df = pd.json_normalize(marvel.results.apply(json.loads))
marvel_final = pd.concat([marvel, marvel_results_df], axis=1)
marvel_final.drop(columns=['results', 'MarvelStory'], inplace=True)
marvel_final.head()

C:\Users\mislentz\AppData\Local\Temp\ipykernel_27536\2848736746.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  marvel['results'] = marvel_results


,FirstName,LastName,Title,BirthDate,EmailAddress,DepartmentName,ExtractedFirstName,ExtractedLastName,ExtractedTitle,ExtractedBirthDate,ExtractedEmailAddress,ExtractedDepartmentName
0,Michael,DeVoe,Sales Store Manager,1966-01-21,stefen0@contoso.com,Production,Michael,DeVoe,Sales Store Manager,1966-01-21,stefen0@contoso.com,Not Found
1,Jerry,Orman,Sales Store Manager,1971-01-26,laura0@contoso.com,Production,Jerry,Orman,Sales Store Manager,1971-01-01,laura0@contoso.com,Production Department
2,Janet,Galore,Sales Store Manager,1965-04-17,wanida0@contoso.com,Marketing,Janet,Galore,Sales Store Manager,1965-04-17,wanida0@contoso.com,Not Found
3,Christen,Anderson,Sales State Manager,1970-12-15,paul0@contoso.com,Production,Christen,Anderson,Sales State Manager,1970-12-15,paul0@contoso.com,Not Found
4,Humberto,Acevedo,Sales Region Manager,1965-01-23,Humberto@contoso.com,Tool Design,Humberto,Acevedo,Sales Region Manager,1965-01-23,Humberto@contoso.com,Tool Design


In [15]:
fairytale_results = await runChain(fairytale_stories, fairytale_stories_chain)
fairytale['results'] = fairytale_results
fairytale_results_df = pd.json_normalize(fairytale.results.apply(json.loads))
fairytale_final = pd.concat([fairytale, fairytale_results_df], axis=1)
fairytale_final.drop(columns=['results', 'FairyTaleStory'], inplace=True)
fairytale_final.head()

C:\Users\mislentz\AppData\Local\Temp\ipykernel_27536\2093501432.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fairytale['results'] = fairytale_results


,FirstName,LastName,Title,BirthDate,EmailAddress,DepartmentName,ExtractedFirstName,ExtractedLastName,ExtractedTitle,ExtractedBirthDate,ExtractedEmailAddress,ExtractedDepartmentName
0,Michael,DeVoe,Sales Store Manager,1966-01-21,stefen0@contoso.com,Production,Michael,DeVoe,Sales Store Manager,1966-01-21,Not Found,Production
1,Jerry,Orman,Sales Store Manager,1971-01-26,laura0@contoso.com,Production,Jerry,Orman,Sales Store Manager,1971-01-26,Not Found,Production
2,Janet,Galore,Sales Store Manager,1965-04-17,wanida0@contoso.com,Marketing,Janet,Galore,Sales Store Manager,1965-04-17,wanida0@contoso.com,Marketing
3,Christen,Anderson,Sales State Manager,1970-12-15,paul0@contoso.com,Production,Christen,Anderson,Sales State Manager,1970-12-15,paul0@contoso.com,Not Found
4,Humberto,Acevedo,Sales Region Manager,1965-01-23,Humberto@contoso.com,Tool Design,Humberto,Acevedo,Sales Region Manager,Not Found,Humberto@contoso.com,Not Found


### Check for Accuracy

Now that we have extracted all of the data, we will compare the extracted fields with the original fields and see the accuracy. We will perform a simple check: if the Expected Field matches the Original Field, then we will +1 to our final score column. The maximum for each row is 6, since we have 6 employee attributes. 

In [16]:
column_pairs = [
    ('FirstName', 'ExtractedFirstName'),
    ('LastName', 'ExtractedLastName'),
    ('Title', 'ExtractedTitle'),
    ('BirthDate', 'ExtractedBirthDate'),
    ('EmailAddress', 'ExtractedEmailAddress'),
    ('DepartmentName', 'ExtractedDepartmentName')
]

def compare_columns_pair(df, col1, col2):
    return df[col1] == df[col2]
def compare_columns_sum(df):
    comparisons = [compare_columns_pair(df, col1, col2) for col1, col2 in column_pairs]
    final_scores = sum(comparisons)
    df['FinalScore'] = final_scores
    df['Error'] = 6 - df['FinalScore']
    return df

In [17]:
compare_columns_sum(marvel_final)

,FirstName,LastName,Title,BirthDate,EmailAddress,DepartmentName,ExtractedFirstName,ExtractedLastName,ExtractedTitle,ExtractedBirthDate,ExtractedEmailAddress,ExtractedDepartmentName,FinalScore,Error
0,Michael,DeVoe,Sales Store Manager,1966-01-21,stefen0@contoso.com,Production,Michael,DeVoe,Sales Store Manager,1966-01-21,stefen0@contoso.com,Not Found,5,1
1,Jerry,Orman,Sales Store Manager,1971-01-26,laura0@contoso.com,Production,Jerry,Orman,Sales Store Manager,1971-01-01,laura0@contoso.com,Production Department,4,2
2,Janet,Galore,Sales Store Manager,1965-04-17,wanida0@contoso.com,Marketing,Janet,Galore,Sales Store Manager,1965-04-17,wanida0@contoso.com,Not Found,5,1
3,Christen,Anderson,Sales State Manager,1970-12-15,paul0@contoso.com,Production,Christen,Anderson,Sales State Manager,1970-12-15,paul0@contoso.com,Not Found,5,1
4,Humberto,Acevedo,Sales Region Manager,1965-01-23,Humberto@contoso.com,Tool Design,Humberto,Acevedo,Sales Region Manager,1965-01-23,Humberto@contoso.com,Tool Design,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Laurent,Bonnet,Sales Store Manager,1976-06-20,margie0@contoso.com,Production,Laurent,Bonnet,Sales Store Manager,1976-06-20,margie0@contoso.com,Not Found,5,1
96,Allan,Guinot,Sales Store Manager,1968-03-13,AllanG@contoso.com,Purchasing,Allan,Guinot,Sales Store Manager,1968-03-13,AllanG@contoso.com,Purchasing Department,5,1
97,Peter,Villadsen,Sales Store Manager,1942-06-13,frank2@contoso.com,Purchasing,Peter,Villadsen,Sales Store Manager,1942-06-13,frank2@contoso.com,Purchasing Department,5,1
98,Vivian,Atlas,Sales Store Manager,1979-07-27,fred0@contoso.com,Production,Vivian,Atlas,Sales Store Manager,1979-07-27,fred0@contoso.com,Not Found,5,1


In [18]:
compare_columns_sum(fairytale_final)

,FirstName,LastName,Title,BirthDate,EmailAddress,DepartmentName,ExtractedFirstName,ExtractedLastName,ExtractedTitle,ExtractedBirthDate,ExtractedEmailAddress,ExtractedDepartmentName,FinalScore,Error
0,Michael,DeVoe,Sales Store Manager,1966-01-21,stefen0@contoso.com,Production,Michael,DeVoe,Sales Store Manager,1966-01-21,Not Found,Production,5,1
1,Jerry,Orman,Sales Store Manager,1971-01-26,laura0@contoso.com,Production,Jerry,Orman,Sales Store Manager,1971-01-26,Not Found,Production,5,1
2,Janet,Galore,Sales Store Manager,1965-04-17,wanida0@contoso.com,Marketing,Janet,Galore,Sales Store Manager,1965-04-17,wanida0@contoso.com,Marketing,6,0
3,Christen,Anderson,Sales State Manager,1970-12-15,paul0@contoso.com,Production,Christen,Anderson,Sales State Manager,1970-12-15,paul0@contoso.com,Not Found,5,1
4,Humberto,Acevedo,Sales Region Manager,1965-01-23,Humberto@contoso.com,Tool Design,Humberto,Acevedo,Sales Region Manager,Not Found,Humberto@contoso.com,Not Found,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Laurent,Bonnet,Sales Store Manager,1976-06-20,margie0@contoso.com,Production,Laurent,Bonnet,Sales Store Manager,1976-06-01,margie0@contoso.com,Department of Production,4,2
96,Allan,Guinot,Sales Store Manager,1968-03-13,AllanG@contoso.com,Purchasing,Allan,Guinot,Sales Store Manager,1968-03-01,Not Found,Purchasing,4,2
97,Peter,Villadsen,Sales Store Manager,1942-06-13,frank2@contoso.com,Purchasing,Peter,Villadsen,Sales Store Manager,1942-06-13,frank2@contoso.com,Purchasing,6,0
98,Vivian,Atlas,Sales Store Manager,1979-07-27,fred0@contoso.com,Production,Vivian,Atlas,Sales Store Manager,1979-07-27,fred0@contoso.com,Production,6,0


Now, we can take our final sums to see the total number of matches made between the extracted data and the real data.

In [19]:
print('The Marvel Stories\n Total Error:', (marvel_final['Error'].sum()), '\n Average Error:', (marvel_final['Error'].mean()), '\n\n')


The Marvel Stories
 Total Error: 97 
 Average Error: 0.97 




In [20]:
print('The Fairytale Stories\n Total Error:', (fairytale_final['Error'].sum()), '\n Average Error:', (fairytale_final['Error'].mean()), '\n\n')

The Fairytale Stories
 Total Error: 116 
 Average Error: 1.16 




### Conclusion

In this exercise we looked at how you can use an LLM to extract data from texts. However, this was a very basic form of analysis. There are still other things that can be explored, for example, which employee attribute was the least identifiable or included. With larger data samples, it also becomes increasingly hard for the LLM to extract valuable pieces of infomation.  